In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import glob
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')
mpl.rcParams['savefig.dpi'] = 128
mpl.rcParams['figure.dpi'] = 128
# Plot size to 14" x 7"
mpl.rc('figure', figsize = (14, 7))
# Font size to 14
mpl.rc('font', size = 14)
# Do not display top and right frame lines
mpl.rc('axes.spines', top = False, right = False)
# Remove grid lines
mpl.rc('axes', grid = False)
# Set backgound color to white
mpl.rc('axes', facecolor = 'white')

In [2]:
# read whole year data
allFiles = glob.glob("data/*.csv")
df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,encoding="utf-8")
    list_.append(df)
df = pd.concat(list_)

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
def transform_cols(df):
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace('_', ' ')
    return df

In [5]:
df = transform_cols(df)

In [6]:
selected_features = ['marketing code','suburb', 'state','post code','enquired',
                     'loan amount','loan reason','property use']
# selected_features = ['marketing code','enquired',
#                      'loan amount','loan reason','property use']
# selected_features = ['marketing code','enquired','loan amount','property use']
target = 'enquiry status';
whole_set = selected_features + [target]

df = df[whole_set]

In [7]:
df = df[~df[target].isin(['In Progress','Just Received','On Hold'])]
df.shape

(252252, 9)

In [8]:
df['loan amount'].fillna(0,inplace=True)

In [9]:
# convert to string to do replacement
df['loan amount'] = df['loan amount'].astype("str")
df['loan amount'] = df['loan amount'].str.replace(",","")

In [10]:
invalid_columns= ['500001-$1000000',
                  '300001-$500000',
                  '0-$300000',
                  '250000 - 300000',
                  '250000-350000',
                  '2600 monthly',
                  'not_sure',
                  '1000,001+',
                 '9999-',
                  'I50000',
                  '1.5 M',
                  '1000001+',
                  '9999-',
                  '80-90k']

In [11]:
df = df[~df['loan amount'].isin(invalid_columns)]
df.shape

(251490, 9)

In [12]:
for _ in df.columns:
    print("The number of null values in: {} == {}".format(_, df[_].isnull().sum()))

The number of null values in: marketing code == 354
The number of null values in: suburb == 59609
The number of null values in: state == 45151
The number of null values in: post code == 19157
The number of null values in: enquired == 0
The number of null values in: loan amount == 0
The number of null values in: loan reason == 22567
The number of null values in: property use == 51944
The number of null values in: enquiry status == 0


In [13]:
df.dropna(axis=0, how='any', inplace=True)
# df = df[~df['marketing code'].isnull()]
# df = df[~df['post code'].isnull()]
# df = df[~df['suburb'].isnull()]
# df = df[~df['loan reason'].isnull()]
# df = df[~df['property use'].isnull()]
df.shape

(172077, 9)

In [14]:
for _ in df.columns:
    print("The number of null values in:{} == {}".format(_, df[_].isnull().sum()))

The number of null values in:marketing code == 0
The number of null values in:suburb == 0
The number of null values in:state == 0
The number of null values in:post code == 0
The number of null values in:enquired == 0
The number of null values in:loan amount == 0
The number of null values in:loan reason == 0
The number of null values in:property use == 0
The number of null values in:enquiry status == 0


In [15]:
drop_values = ['QLD','victoria',"270-1176","VIC","2575b","6110r"]
df = df[~df['post code'].isin(drop_values)]

In [16]:
def transform(df): 
    df['loan amount'] = df['loan amount'].astype('float')
    df['enquired'] = pd.DatetimeIndex(df['enquired'])
    df['month'] = df['enquired'].dt.month
    df['day'] = df['enquired'].dt.day
    df['hour'] = df['enquired'].dt.hour
    df['weekday'] = df['enquired'].dt.dayofweek
    df['post code'] = df['post code'].astype('int')
    
    if 'enquired'in df.columns:
        df.drop(['enquired'], axis = 1, inplace = True)

In [17]:
transform(df)

ValueError: invalid literal for int() with base 10: '6110r'

In [ ]:
df.to_csv("2013-2017.csv",index=False)